In [6]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
%matplotlib inline

import torch
from torch import nn, optim
from torch import autograd
import torch.nn.functional as F
from torch.nn import Parameter

from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader, Subset
from PIL import Image, ImageOps, ImageEnhance

import cv2
import albumentations as A
# from albumentations.pytorch import ToTensor

import glob
import xml.etree.ElementTree as ET 
import shutil
from tqdm import tqdm
import time
import random


import os 
print(os.listdir('../data/generative-dog-images'))

['all-dogs.zip', 'Annotation.zip', 'all-dogs', 'Annotation']


In [7]:
TIME_LIMIT = 32400 - 60 * 10
start_time = time.time()

def elapsed_time(start_time):
    return time.time() - start_time

In [8]:
# random seeds
seed = 2019
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

BATCH_SIZE = 32
NUM_WORKERS = 4
EMA = False
LABEL_NOISE = False
LABEL_NOISE_PROB = 0.1

# Pre-Processing

In [10]:
PATH = '../data/generative-dog-images/all-dogs/all-dogs'
img_filnames = os.listdir(PATH)
len(img_filnames)

20579

In [13]:
PATH_ANNOTATION = '../data/generative-dog-images/Annotation/Annotation/'
breeds = glob.glob(PATH_ANNOTATION + '*')
annotations = []
for breed in breeds:
    annotations += glob.glob(breed + '/*')
len(annotations)

20580

In [15]:
breed_map = {}
for annotation in annotations:
    breed = annotation.split('/')[-2]
    index = breed.split('-')[0]
    breed_map.setdefault(index, breed)

n_classes = len(breed_map)
n_classes

120